# differentiable volumetric rendering
## Reconstructing 3D models from multi-view images

在本節中，我們將展示一個使用可微體積渲染從多視圖影像重建 3D 模型的範例


Of course, to reconstruct the 3D world, we need multiple images from multiple views.


In [1]:
import sys
import torch
pyt_version = torch.__version__.split('+')[0].replace(".","")
version_str = "".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace('.',''),
    f"_pyt{pyt_version}"
])
!pip install fvcore iopath
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=bd9ac7daa4e021263694e297818ed5920fc11d28632a0084ad60af8af19b5649
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31530 sha256=c87244bc08e23e84adaa5da1bedf880c7329ac7ebaab2233063bd8f5c2b9a00f
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu118_pyt201/download.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 69.0 MB/s eta 0:00:00


In [2]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.
#
# This source code is licensed under the BSD-style license found in the
# LICENSE file in the root directory of this source tree.

import matplotlib.pyplot as plt


def image_grid(
    images,
    rows=None,
    cols=None,
    fill: bool = True,
    show_axes: bool = False,
    rgb: bool = True,
):
    """
    A util function for plotting a grid of images.

    Args:
        images: (N, H, W, 4) array of RGBA images
        rows: number of rows in the grid
        cols: number of columns in the grid
        fill: boolean indicating if the space between images should be filled
        show_axes: boolean indicating if the axes of the plots should be visible
        rgb: boolean, If True, only RGB channels are plotted.
            If False, only the alpha channel is plotted.

    Returns:
        None
    """
    if (rows is None) != (cols is None):
        raise ValueError("Specify either both rows and cols or neither.")

    if rows is None:
        rows = len(images)
        cols = 1

    gridspec_kw = {"wspace": 0.0, "hspace": 0.0} if fill else {}
    fig, axarr = plt.subplots(rows, cols, gridspec_kw=gridspec_kw, figsize=(15, 9))
    bleed = 0
    fig.subplots_adjust(left=bleed, bottom=bleed, right=(1 - bleed), top=(1 - bleed))

    for ax, im in zip(axarr.ravel(), images):
        if rgb:
            # only render RGB channels
            ax.imshow(im[..., :3])
        else:
            # only render Alpha channel
            ax.imshow(im[..., 3])
        if not show_axes:
            ax.set_axis_off()

In [3]:
!pwd

/content


In [4]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.
#
# This source code is licensed under the BSD-style license found in the
# LICENSE file in the root directory of this source tree.

import os

import numpy as np
import torch
from pytorch3d.io import load_objs_as_meshes
from pytorch3d.renderer import (
    BlendParams,
    FoVPerspectiveCameras,
    MeshRasterizer,
    MeshRenderer,
    PointLights,
    RasterizationSettings,
    SoftPhongShader,
    SoftSilhouetteShader,
    look_at_view_transform,
)


# create the default data directory
# current_dir = os.path.dirname(os.path.realpath(__file__))
current_dir = '/content'
DATA_DIR = os.path.join(current_dir, ".", "data", "cow_mesh")


def generate_cow_renders(
    num_views: int = 40, data_dir: str = DATA_DIR, azimuth_range: float = 180
):
    """
    This function generates `num_views` renders of a cow mesh.
    The renders are generated from viewpoints sampled at uniformly distributed
    azimuth intervals. The elevation is kept constant so that the camera's
    vertical position coincides with the equator.

    For a more detailed explanation of this code, please refer to the
    docs/tutorials/fit_textured_mesh.ipynb notebook.

    Args:
        num_views: The number of generated renders.
        data_dir: The folder that contains the cow mesh files. If the cow mesh
            files do not exist in the folder, this function will automatically
            download them.

    Returns:
        cameras: A batch of `num_views` `FoVPerspectiveCameras` from which the
            images are rendered.
        images: A tensor of shape `(num_views, height, width, 3)` containing
            the rendered images.
        silhouettes: A tensor of shape `(num_views, height, width)` containing
            the rendered silhouettes.
    """

    # set the paths

    # download the cow mesh if not done before
    cow_mesh_files = [
        os.path.join(data_dir, fl) for fl in ("cow.obj", "cow.mtl", "cow_texture.png")
    ]
    if any(not os.path.isfile(f) for f in cow_mesh_files):
        os.makedirs(data_dir, exist_ok=True)
        os.system(
            f"wget -P {data_dir} "
            + "https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow.obj"
        )
        os.system(
            f"wget -P {data_dir} "
            + "https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow.mtl"
        )
        os.system(
            f"wget -P {data_dir} "
            + "https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow_texture.png"
        )

    # Setup
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        torch.cuda.set_device(device)
    else:
        device = torch.device("cpu")

    # Load obj file
    obj_filename = os.path.join(data_dir, "cow.obj")
    mesh = load_objs_as_meshes([obj_filename], device=device)

    # We scale normalize and center the target mesh to fit in a sphere of radius 1
    # centered at (0,0,0). (scale, center) will be used to bring the predicted mesh
    # to its original center and scale.  Note that normalizing the target mesh,
    # speeds up the optimization but is not necessary!
    verts = mesh.verts_packed()
    N = verts.shape[0]
    center = verts.mean(0)
    scale = max((verts - center).abs().max(0)[0])
    mesh.offset_verts_(-(center.expand(N, 3)))
    mesh.scale_verts_((1.0 / float(scale)))

    # Get a batch of viewing angles.
    elev = torch.linspace(0, 0, num_views)  # keep constant
    azim = torch.linspace(-azimuth_range, azimuth_range, num_views) + 180.0

    # Place a point light in front of the object. As mentioned above, the front of
    # the cow is facing the -z direction.
    lights = PointLights(device=device, location=[[0.0, 0.0, -3.0]])

    # Initialize an OpenGL perspective camera that represents a batch of different
    # viewing angles. All the cameras helper methods support mixed type inputs and
    # broadcasting. So we can view the camera from the a distance of dist=2.7, and
    # then specify elevation and azimuth angles for each viewpoint as tensors.
    R, T = look_at_view_transform(dist=2.7, elev=elev, azim=azim)
    cameras = FoVPerspectiveCameras(device=device, R=R, T=T)

    # Define the settings for rasterization and shading. Here we set the output
    # image to be of size 128X128. As we are rendering images for visualization
    # purposes only we will set faces_per_pixel=1 and blur_radius=0.0. Refer to
    # rasterize_meshes.py for explanations of these parameters.  We also leave
    # bin_size and max_faces_per_bin to their default values of None, which sets
    # their values using heuristics and ensures that the faster coarse-to-fine
    # rasterization method is used.  Refer to docs/notes/renderer.md for an
    # explanation of the difference between naive and coarse-to-fine rasterization.
    raster_settings = RasterizationSettings(
        image_size=128, blur_radius=0.0, faces_per_pixel=1
    )

    # Create a Phong renderer by composing a rasterizer and a shader. The textured
    # Phong shader will interpolate the texture uv coordinates for each vertex,
    # sample from a texture image and apply the Phong lighting model
    blend_params = BlendParams(sigma=1e-4, gamma=1e-4, background_color=(0.0, 0.0, 0.0))
    renderer = MeshRenderer(
        rasterizer=MeshRasterizer(cameras=cameras, raster_settings=raster_settings),
        shader=SoftPhongShader(
            device=device, cameras=cameras, lights=lights, blend_params=blend_params
        ),
    )

    # Create a batch of meshes by repeating the cow mesh and associated textures.
    # Meshes has a useful `extend` method which allows us do this very easily.
    # This also extends the textures.
    meshes = mesh.extend(num_views)

    # Render the cow mesh from each viewing angle
    target_images = renderer(meshes, cameras=cameras, lights=lights)

    # Rasterization settings for silhouette rendering
    sigma = 1e-4
    raster_settings_silhouette = RasterizationSettings(
        image_size=128, blur_radius=np.log(1.0 / 1e-4 - 1.0) * sigma, faces_per_pixel=50
    )

    # Silhouette renderer
    renderer_silhouette = MeshRenderer(
        rasterizer=MeshRasterizer(
            cameras=cameras, raster_settings=raster_settings_silhouette
        ),
        shader=SoftSilhouetteShader(),
    )

    # Render silhouette images.  The 3rd channel of the rendering output is
    # the alpha/silhouette channel
    silhouette_images = renderer_silhouette(meshes, cameras=cameras, lights=lights)

    # binary silhouettes
    silhouette_binary = (silhouette_images[..., 3] > 1e-4).float()

    return cameras, target_images[..., :3], silhouette_binary

In [5]:
import os
import sys
import time
import json
import glob
import torch
import math
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from pytorch3d.structures import Volumes
from pytorch3d.renderer import (
    FoVPerspectiveCameras,
    VolumeRenderer,
    NDCGridRaysampler,
    EmissionAbsorptionRaymarcher
)
from pytorch3d.transforms import so3_exp_map
# from plot_image_grid import image_grid  # 直接在上面
# from generate_cow_renders import generate_cow_renders # 如上文

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

產生40張相機圖片，不同角度的剪影影像。

In [7]:
target_cameras,target_images ,target_silhouettes = generate_cow_renders(num_views=40)

In [8]:
generate_cow_renders(num_views=40)

(FoVPerspectiveCameras(),
 tensor([[[[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           ...,
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],
 
          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           ...,
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],
 
          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           ...,
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],
 
          ...,
 
          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           ...,
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],
 
          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           ...,
           [0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.]],
 
          [[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 0.],
           ...,
           [0., 0., 0

接下來，我們定義一個光線採樣器。 正如我們在前面幾節中討論的，光線採樣器用於採樣光線，以及每條光線的點數：

In [9]:
render_size = 128
volume_extent_world = 3.0
raysampler = NDCGridRaysampler(
    image_width = render_size,
    image_height=render_size,
    n_pts_per_ray = 150,
    min_depth = 0.1 ,
    max_depth = volume_extent_world,
)

接下來，我們像以前一樣創建光線行進器。 注意，這次，我們定義了一個 VolumeRenderer 類型的變數渲染器。 VolumeRenderer 只是一個很好的介面，光線採樣器和光線行進器在後台完成所有繁重的工作：

In [10]:
raymarcher = EmissionAbsorptionRaymarcher()
renderer = VolumeRenderer(
    raysampler=raysampler,raymarcher=raymarcher
)

接下來，我們定義一個 VolumeModel 類別。 此類別僅用於封裝體積，以便可以在前向函數中計算梯度，並且最佳化器可以更新體積密度和顏色：

In [11]:
class VolumeModel(torch.nn.Module):
  def __init__(self,renderer,volume_size=[64]*3,voxel_size=0.1):
    super().__init__()
    self.log_densities = torch.nn.Parameter(-4.0*torch.ones(1,*volume_size))
    self.log_colors = torch.nn.Parameter(torch.zeros(3,*volume_size))
    self._voxel_size = voxel_size
    self._renderer = renderer

  def forward(self,cameras):
    batch_size = cameras.R.shape[0]
    densities = torch.sigmoid(self.log_densities)
    colors = torch.sigmoid(self.log_colors)
    volumes = Volumes(
        densities==densities[None].expand(
            batch_size,*self.log_densities.shape
        ),features=colors[None].expand(batch_size,*self.log_colors.shape),
        voxel_size=self._voxel_size,
    )
    return self._renderer(cameras=cameras,volumes=volumes)[0]

定義 Huber 損失函數。 Huber 損失函數是一種穩健的損失函數，可防止少數異常值拖累最佳化遠離真正的最佳解。 最小化此損失函數將使 x 更接近 y：

In [12]:
def huber(x,y,scaling=0.1):
  diff_sq = (x-y)**2
  loss = ((1+diff_sq/(scaling**2)).clamp(1e-4).sqrt() -1 ) * float(scaling)
  return loss



In [13]:
target_cameras = target_cameras.to(device)
target_images = target_images.to(device)
target_silhouettes = target_silhouettes.to(device)
volume_size = 128
# volume_model = VolumeModel(
#     renderer,
#     volume_size=[volume_size]*3,
#     voxel_size=volume_extent_world/volume_size,
# ).to(device)

volume_model = VolumeModel(
    renderer,
    volume_size=[volume_size] * 3,
    voxel_size=volume_extent_world / volume_size,
).to(device)

In [14]:
lr = 0.1
optimizer = torch.optim.Adam(volume_model.parameters(),lr=lr)
batch_size = 10
n_iter = 300

In [17]:
from pytorch3d.renderer.points.pulsar.unified import FoVOrthographicCameras
for iteration in range(n_iter):
  if iteration == round(n_iter*0.75):
    print('Decreasing LR 10 -fold')
    optimizer = torch.optim.Adam(volume_model.parameters(),lr=lr*0.1)
  optimizer.zero_grad()
  batch_idx = torch.randperm(len(target_cameras))[:batch_size]
  # sample the minibatch of cameras
  batch_cameras = FoVPerspectiveCameras(
      R = target_cameras.R[batch_idx],
      T = target_cameras.T[batch_idx],
      znear = target_cameras.znear[batch_idx],
      zfar = target_cameras.zfar[batch_idx],
      aspect_ratio = target_cameras.aspect_ratio[batch_idx],
      fov = target_cameras.fov[batch_idx],
      device=device
  )
  rendered_images ,rendered_silhouettes = volume_model(
      batch_cameras
  ).split([3,1],dim=-1)
  # rendered_images, rendered_silhouettes = volume_model(
  #       batch_cameras
  #   ).split([3, 1], dim=-1)


  sil_err = huber(rendered_silhouettes[...,0],target_silhouettes[batch_idx],).abs().mean()
  color_err = huber(
      rendered_images,target_images[batch_idx]
  ).abs().mean()

  loss = color_err + sil_err
  loss.backward()
  optimizer.step()
  # with torch.no_grad():
    # rotating_volume_frames = generate_roa

RuntimeError: ignored

In [19]:
def generate_rotating_volume(volume_model, n_frames=50):
    logRs = torch.zeros(n_frames, 3, device=device)
    logRs[:, 1] = torch.linspace(0.0, 2.0 * 3.14, n_frames, device=device)
    Rs = so3_exp_map(logRs)
    Ts = torch.zeros(n_frames, 3, device=device)
    Ts[:, 2] = 2.7
    frames = []
    print('Generating rotating volume ...')
    for R, T in zip(Rs, Ts):
        camera = FoVPerspectiveCameras(
            R=R[None],
            T=T[None],
            znear=target_cameras.znear[0],
            zfar=target_cameras.zfar[0],
            aspect_ratio=target_cameras.aspect_ratio[0],
            fov=target_cameras.fov[0],
            device=device,
        )
        frames.append(volume_model(camera)[..., :3].clamp(0.0, 1.0))
    return torch.cat(frames)


In [18]:
with torch.no_grad():
  rotating_volume_frames = generate_rotating_volume(volume_model,n_frames=7*4)
image_grid(rotating_volume_frames.clamp(0.,1..cpu().numpy(),rows=4,cols=7,rgb=True,fill=True))

NameError: ignored